In [ ]:
import cv2
import os

# Create a VideoCapture object
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Define the signs for which we want to capture images
signs = [chr(i) for i in range(ord('A'), ord('Z') + 1)] + [str(i) for i in range(0, 10)]

# Number of images to capture for each sign
num_images_per_sign = 100

# Create a common directory for all signs if it does not exist
os.makedirs('data', exist_ok=True)

# Capture images at a faster rate
frame_interval = 3  # Capture every 3 frames
frame_count = 0

# Initialize sign capture
current_sign = None
capturing = False

# Capture loop
while True:
    ret, frame = cap.read()

    if ret:
        if capturing:
            frame_count += 1
            if frame_count % frame_interval == 0:
                if current_sign is not None:
                    counter = len(os.listdir(f'data/{current_sign}'))
                    if counter < num_images_per_sign:
                        cv2.imwrite(f'data/{current_sign}/{counter}.jpg', frame)
                        print(f"Captured image {counter}/{num_images_per_sign}")
                    else:
                        capturing = False
                        print(
                            f"Finished capturing images for sign {current_sign}. Press 's' to continue with the next sign or 'q' to quit.")

        if current_sign is not None and capturing:
            note = f"Capturing {current_sign} - Press 's' to stop"
            cv2.putText(frame, note, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow('frame', frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            if not capturing:
                if signs:
                    current_sign = signs.pop(0)  # Get the next sign to capture
                    os.makedirs(f'data/{current_sign}', exist_ok=True)
                    frame_count = 0
                    capturing = True
                    print(f"Start capturing images for sign {current_sign}. Press 's' to stop.")
                else:
                    print("Finished capturing images for all signs. Press 'q' to quit.")
            else:
                capturing = False
                print(
                    f"Stopped capturing images for sign {current_sign}. Press 's' to continue with the next sign or 'q' to quit.")
        elif key == ord('q'):
            break
    else:
        print("Error: Could not read frame from camera.")
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()